In [15]:
import ast
import datetime
import os
import random
import warnings

import autogluon.tabular
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection

pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 200

warnings.filterwarnings("ignore", message="Can't initialize NVML")

In [16]:
DIRECTORY_DATA = 'data/model-autogluon'
os.makedirs(DIRECTORY_DATA, exist_ok=True)

# Preparing data

In [17]:
# FILENAME_DF_ALL_CLEAN = None
#FILENAME_DF_ALL_CLEAN = 'data/20241230_173555/df_all_clean.csv'
# FILENAME_DF_ALL_CLEAN = 'data/20250128_094430/df_all_clean.csv'
# FILENAME_DF_ALL_CLEAN = 'data/20250220_094622_halfway/df_all_clean.csv'
FILENAME_DF_ALL_CLEAN = 'data/20250220_094622/df_all_clean.csv'

In [18]:
SEPARATOR_COL = ': '


def col2parts(col: str) -> tuple[str, str]:
    parts = col.split(SEPARATOR_COL)
    if len(parts) == 1:
        return '', parts[0]
    assert len(parts) == 2
    return parts[0], parts[1]


if FILENAME_DF_ALL_CLEAN is not None:
    df_all = pd.read_csv(FILENAME_DF_ALL_CLEAN, header=list(range(2)), index_col=0)
    df_all.columns = [SEPARATOR_COL.join(col) for col in df_all.columns]
else:
    runnames = [
        '20241203_170129_all600',
        '20241213_104400_racing',
        '20241214_122216_racing_passhum',
    ]
    df_all = pd.concat(
        [pd.read_csv(f'data/{runname}/df_all.csv') 
         for runname in runnames],
        keys=runnames,
        names=['runname', 'row']
    )
    assert all(SEPARATOR_COL not in col for col in df_all.columns), df_all.columns

print(FILENAME_DF_ALL_CLEAN)
df_all.info()

data/20250220_094622/df_all_clean.csv
<class 'pandas.core.frame.DataFrame'>
Index: 2387 entries, map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing 50% stops to map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops
Columns: 438 entries, Violation type: Priority violation to Output of simulation (execution): Near-miss rate
dtypes: bool(6), float64(408), int64(22), object(2)
memory usage: 7.9+ MB


In [19]:
# df_all = df_all[~df_all.index.str.contains('50% stops')]
# df_all.info()

In [20]:
def series2values(series):
    dtype = series.dtype
    if dtype == 'bool':
        return series.astype('int').values
    if dtype in ('int64', 'float64'):
        return series.values
    if dtype == 'object':
        return series.astype('category').cat.codes
    raise TypeError(f'{dtype} is not supported')
    

if FILENAME_DF_ALL_CLEAN is not None:
    COLUMNS_NONSPLIT = {
        col: series2values
        for col in [
            'Static map features: ~ Position',  # includes i_map, i_position
        ]
    }
    COLUMNS_INPUT = {col: series2values 
                     # for col in (
                     #    'Static map features: Connectivity',
                     # )}
                     for col in df_all.columns 
                     if col2parts(col)[0] != 'Output of simulation (execution)'
                     and not col2parts(col)[1].startswith('~ ')}
    # COLUMNS_INPUT.pop('Static map features: No. of OPs')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Mission length for MV (V0)')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Mission length for AV (V1)')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Mission length for AV (V2)')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Mission length for AV (V3)')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Total Mission length for AVs (V1-V3)')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Total Mission length for AVs (V1-V3)')  # TODO
    # COLUMNS_INPUT = {key: value for key, value in COLUMNS_INPUT.items()
    #                  if 'No. of OPs' not in key and 'POD' not in key}   
    COLUMNS_INPUT = {key: value for key, value in COLUMNS_INPUT.items()
                     if 'POD C' not in key}   
    COLUMNS_OUTPUT = {col: series2values
                      for col in (
                        'Output of simulation (execution): Total No. of completed missions for AVs (V1-V3)',
                        'Output of simulation (execution): Collision rate'
                      )}
                      # for col in df_all.columns 
                      # if col2parts(col)[0] == 'Output of simulation (execution)'
                      # and not col2parts(col)[1].startswith('~ ')}
else:
    COLUMNS_NONSPLIT = {
        'Scenario ID': lambda series: series.values,
    }
    COLUMNS_INPUT = {
        'Coordination strategy': lambda series: series.astype('category').cat.codes.values,
        'isCanPassFirstHum': lambda series: series.astype('int').values,
        'isRacingThroughCrossroadAllowed': lambda series: series.astype('int').values,
        
        'Vehicle ID': lambda series: series.values,
        'Linearization C': lambda series: [
            series.apply(lambda x: -1 if x is None else x[i])
            for i in range(len(series.dropna().iloc[0]))
        ],   
    }
    COLUMNS_OUTPUT = {
        'traveled total, m': lambda series: series.values,
        'No. of completed missions': lambda series: series.values,
        'No. of collisions': lambda series: series.values,
        'No. of near-misses': lambda series: series.values,
    }

COLUMNS_ALL = {**COLUMNS_NONSPLIT, **COLUMNS_INPUT, **COLUMNS_OUTPUT}
COLUMNS_ALL

{'Static map features: ~ Position': <function __main__.series2values(series)>,
 'Violation type: Priority violation': <function __main__.series2values(series)>,
 'Violation type: Speed violation': <function __main__.series2values(series)>,
 'Violation type: Priority violation and Speed violation': <function __main__.series2values(series)>,
 'Coordination strategy: Change of priorities': <function __main__.series2values(series)>,
 'Coordination strategy: Stops': <function __main__.series2values(series)>,
 'Coordination strategy: Rerouting': <function __main__.series2values(series)>,
 'Static map features: No. of OPs': <function __main__.series2values(series)>,
 'Static map features: Connectivity': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD score for MV (V0)': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD score for AV (V1)': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD score fo

In [21]:
df_inout = df_all[list(COLUMNS_ALL)]
df_inout

,Static map features: ~ Position,Violation type: Priority violation,Violation type: Speed violation,Violation type: Priority violation and Speed violation,Coordination strategy: Change of priorities,Coordination strategy: Stops,Coordination strategy: Rerouting,Static map features: No. of OPs,Static map features: Connectivity,Output of simulation (planning): POD score for MV (V0),Output of simulation (planning): POD score for AV (V1),Output of simulation (planning): POD score for AV (V2),Output of simulation (planning): POD score for AV (V3),Output of simulation (planning): Mean POD score for AVs (V1-V3),Output of simulation (planning): Mission length for MV (V0),Output of simulation (planning): Mission length for AV (V1),Output of simulation (planning): Mission length for AV (V2),Output of simulation (planning): Mission length for AV (V3),Output of simulation (planning): Total Mission length for AVs (V1-V3),Output of simulation (planning): Mean Mission length for AVs (V1-V3),Output of simulation (execution): Total No. of completed missions for AVs (V1-V3),Output of simulation (execution): Collision rate
"map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing 50% stops",1-1,True,False,False,False,False,False,2,high,0.195,0.104,0.058,0.149,0.115,721,227,547,1017,1791,597.000,32,0.400
"map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities",1-1,True,False,False,True,False,False,2,high,0.195,0.104,0.058,0.149,0.115,721,227,547,1017,1791,597.000,33,0.235
"map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing ignoring human",1-1,True,False,False,False,False,False,2,high,0.195,0.104,0.058,0.149,0.115,721,227,547,1017,1791,597.000,36,0.318
"map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing no",1-1,False,False,False,False,False,False,2,high,0.195,0.104,0.058,0.149,0.115,721,227,547,1017,1791,597.000,6,0.000
"map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing stops",1-1,True,False,False,False,True,False,2,high,0.195,0.104,0.058,0.149,0.115,721,227,547,1017,1791,597.000,32,0.417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing 50% stops",9-9,False,False,True,False,False,False,1,low,0.670,0.652,0.348,0.033,0.406,744,1057,808,572,2437,812.333,12,0.667
"map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing change of priorities",9-9,False,False,True,True,False,False,1,low,0.670,0.652,0.348,0.033,0.406,744,1057,808,572,2437,812.333,13,0.333
"map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing ignoring human",9-9,False,False,True,False,False,False,1,low,0.670,0.652,0.348,0.033,0.406,744,1057,808,572,2437,812.333,21,2.000
"map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing no",9-9,False,True,False,False,False,False,1,low,0.670,0.652,0.348,0.033,0.406,744,1057,808,572,2437,812.333,17,0.000


In [22]:
def parse_tuple_string(tuple_string):
    if pd.isna(tuple_string):
        return None
    return ast.literal_eval(tuple_string)


def preprocess_inout(df_inout):
    df_inout = df_inout.copy()
    
    linearization_columns = [col for col in df_inout.columns if isinstance(col, str) and col.startswith('Linearization')]
    for col in linearization_columns:
        df_inout[col] = df_inout[col].apply(parse_tuple_string)
        
    dict_preprocessed = {}
    for col, series2data in COLUMNS_ALL.items():
        data = series2data(df_inout[col])
        if not isinstance(data, list):
            dict_preprocessed[col] = data
        else:
            for i, series in enumerate(data):
                dict_preprocessed[f'{col}#{i}'] = series
    
    return pd.DataFrame(dict_preprocessed)
    

df_preprocessed = preprocess_inout(df_inout)
df_preprocessed

,Static map features: ~ Position,Violation type: Priority violation,Violation type: Speed violation,Violation type: Priority violation and Speed violation,Coordination strategy: Change of priorities,Coordination strategy: Stops,Coordination strategy: Rerouting,Static map features: No. of OPs,Static map features: Connectivity,Output of simulation (planning): POD score for MV (V0),Output of simulation (planning): POD score for AV (V1),Output of simulation (planning): POD score for AV (V2),Output of simulation (planning): POD score for AV (V3),Output of simulation (planning): Mean POD score for AVs (V1-V3),Output of simulation (planning): Mission length for MV (V0),Output of simulation (planning): Mission length for AV (V1),Output of simulation (planning): Mission length for AV (V2),Output of simulation (planning): Mission length for AV (V3),Output of simulation (planning): Total Mission length for AVs (V1-V3),Output of simulation (planning): Mean Mission length for AVs (V1-V3),Output of simulation (execution): Total No. of completed missions for AVs (V1-V3),Output of simulation (execution): Collision rate
"map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing 50% stops",0,1,0,0,0,0,0,2,0,0.195,0.104,0.058,0.149,0.115,721,227,547,1017,1791,597.000,32,0.400
"map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities",0,1,0,0,1,0,0,2,0,0.195,0.104,0.058,0.149,0.115,721,227,547,1017,1791,597.000,33,0.235
"map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing ignoring human",0,1,0,0,0,0,0,2,0,0.195,0.104,0.058,0.149,0.115,721,227,547,1017,1791,597.000,36,0.318
"map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing no",0,0,0,0,0,0,0,2,0,0.195,0.104,0.058,0.149,0.115,721,227,547,1017,1791,597.000,6,0.000
"map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing stops",0,1,0,0,0,1,0,2,0,0.195,0.104,0.058,0.149,0.115,721,227,547,1017,1791,597.000,32,0.417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing 50% stops",99,0,0,1,0,0,0,1,1,0.670,0.652,0.348,0.033,0.406,744,1057,808,572,2437,812.333,12,0.667
"map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing change of priorities",99,0,0,1,1,0,0,1,1,0.670,0.652,0.348,0.033,0.406,744,1057,808,572,2437,812.333,13,0.333
"map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing ignoring human",99,0,0,1,0,0,0,1,1,0.670,0.652,0.348,0.033,0.406,744,1057,808,572,2437,812.333,21,2.000
"map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing no",99,0,1,0,0,0,0,1,1,0.670,0.652,0.348,0.033,0.406,744,1057,808,572,2437,812.333,17,0.000


In [23]:
def show(obj, title=None):
    if title is not None:
        display(HTML(f"<h3>{title}</h3>"))
    display(obj)


def shuffle_df(df):
    """
    # Shuffle by Scenario ID
    unique_scenarios = df['Scenario ID'].unique()  # Get unique Scenario IDs
    shuffled_scenarios = pd.Series(unique_scenarios).sample(frac=1, random_state=1).tolist()  # Shuffle Scenario IDs
    
    # Reorder the dataframe by the shuffled Scenario IDs
    df_shuffled = pd.concat([df[df['Scenario ID'] == scenario] for scenario in shuffled_scenarios])
    
    # Reset index (optional)
    df_shuffled = df_shuffled.reset_index(drop=True)
    
    return df_shuffled
    """
    return df.sample(frac=1)


def split_train_test(df):
    test_size = 0.2
    random_state = 1
    
    if not COLUMNS_NONSPLIT:
        return sklearn.model_selection.train_test_split(df, test_size=test_size, random_state=random_state)
    
    gss = sklearn.model_selection.GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    column, = list(COLUMNS_NONSPLIT)
    groups = df[column]
    
    # Split the data
    for train_idx, test_idx in gss.split(df, groups=groups):
        return shuffle_df(df.iloc[train_idx]), shuffle_df(df.iloc[test_idx])


df_train, df_test = split_train_test(df_preprocessed)
show(df_train, 'df_train')
show(df_test, 'df_test')
if 'Static map features: ~ Position' in df_train.columns:
    assert not set(df_train['Static map features: ~ Position']) & set(df_test['Static map features: ~ Position']) 

,Static map features: ~ Position,Violation type: Priority violation,Violation type: Speed violation,Violation type: Priority violation and Speed violation,Coordination strategy: Change of priorities,Coordination strategy: Stops,Coordination strategy: Rerouting,Static map features: No. of OPs,Static map features: Connectivity,Output of simulation (planning): POD score for MV (V0),Output of simulation (planning): POD score for AV (V1),Output of simulation (planning): POD score for AV (V2),Output of simulation (planning): POD score for AV (V3),Output of simulation (planning): Mean POD score for AVs (V1-V3),Output of simulation (planning): Mission length for MV (V0),Output of simulation (planning): Mission length for AV (V1),Output of simulation (planning): Mission length for AV (V2),Output of simulation (planning): Mission length for AV (V3),Output of simulation (planning): Total Mission length for AVs (V1-V3),Output of simulation (planning): Mean Mission length for AVs (V1-V3),Output of simulation (execution): Total No. of completed missions for AVs (V1-V3),Output of simulation (execution): Collision rate
"map-generator/generated-maps/3_without_bridges/scenario7-7.json, passhum 0, slowness without rerouting, forcing 50% stops",77,0,0,1,0,0,0,1,1,0.774,0.178,0.697,0.858,0.666,454,612,1159,1373,3144,1048.000,6,0.429
"map-generator/generated-maps/3_with_bridges/scenario7-9.json, passhum 0, slowness with rerouting, forcing no",79,0,1,0,0,0,1,1,0,0.515,0.069,0.381,0.637,0.438,455,537,1076,1302,2915,971.667,7,0.000
"map-generator/generated-maps/3_with_bridges/scenario9-10.json, passhum 0, slowness with rerouting, forcing 50% stops",91,0,0,1,0,0,1,1,0,0.617,0.525,0.038,0.040,0.279,754,1070,525,572,2167,722.333,11,0.333
"map-generator/generated-maps/3_without_bridges/scenario1-7.json, passhum 0, slowness no, forcing 50% stops",7,1,0,0,0,0,0,2,1,0.193,0.032,0.076,0.176,0.114,658,355,619,858,1832,610.667,26,1.200
"map-generator/generated-maps/3_with_bridges/scenario1-9.json, passhum 0, slowness without rerouting, forcing 50% stops",9,0,0,1,0,0,0,2,0,0.094,0.145,0.109,0.021,0.076,721,227,382,523,1132,377.333,39,0.333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"map-generator/generated-maps/3_with_bridges/scenario7-3.json, passhum 0, slowness no, forcing 50% stops",73,1,0,0,0,0,0,1,0,0.171,0.029,0.064,0.269,0.158,455,465,537,1000,2002,667.333,21,0.857
"map-generator/generated-maps/3_without_bridges/scenario10-10.json, passhum 0, slowness no, forcing no",11,0,0,0,0,0,0,2,1,0.209,0.200,0.295,0.008,0.204,714,402,824,375,1601,533.667,31,0.000
"map-generator/generated-maps/3_without_bridges/scenario6-6.json, passhum 0, slowness no, forcing 50% stops",66,1,0,0,0,0,0,2,1,0.190,0.166,0.022,0.031,0.091,564,877,491,530,1898,632.667,23,0.667
"map-generator/generated-maps/3_with_bridges/scenario4-8.json, passhum 0, slowness no, forcing ignoring human",48,1,0,0,0,0,0,1,0,0.233,0.034,0.066,0.217,0.130,382,513,583,996,2092,697.333,22,0.400


,Static map features: ~ Position,Violation type: Priority violation,Violation type: Speed violation,Violation type: Priority violation and Speed violation,Coordination strategy: Change of priorities,Coordination strategy: Stops,Coordination strategy: Rerouting,Static map features: No. of OPs,Static map features: Connectivity,Output of simulation (planning): POD score for MV (V0),Output of simulation (planning): POD score for AV (V1),Output of simulation (planning): POD score for AV (V2),Output of simulation (planning): POD score for AV (V3),Output of simulation (planning): Mean POD score for AVs (V1-V3),Output of simulation (planning): Mission length for MV (V0),Output of simulation (planning): Mission length for AV (V1),Output of simulation (planning): Mission length for AV (V2),Output of simulation (planning): Mission length for AV (V3),Output of simulation (planning): Total Mission length for AVs (V1-V3),Output of simulation (planning): Mean Mission length for AVs (V1-V3),Output of simulation (execution): Total No. of completed missions for AVs (V1-V3),Output of simulation (execution): Collision rate
"map-generator/generated-maps/3_with_bridges/scenario3-2.json, passhum 0, slowness with rerouting, forcing 50% stops",32,0,0,1,0,0,1,1,0,0.571,0.705,0.421,0.046,0.456,705,906,482,511,1899,633.000,19,0.400
"map-generator/generated-maps/3_without_bridges/scenario6-5.json, passhum 0, slowness no, forcing ignoring human",65,1,0,0,0,0,0,2,1,0.110,0.044,0.099,0.042,0.061,566,327,477,637,1441,480.333,31,0.444
"map-generator/generated-maps/3_without_bridges/scenario5-10.json, passhum 0, slowness no, forcing stops",51,1,0,0,0,1,0,1,1,0.920,0.187,0.532,1.034,0.700,627,607,1029,1449,3085,1028.333,10,0.882
"map-generator/generated-maps/3_with_bridges/scenario4-4.json, passhum 0, slowness no, forcing ignoring human",44,1,0,0,0,0,0,1,0,0.296,0.188,0.065,0.216,0.170,560,711,561,996,2268,756.000,21,0.765
"map-generator/generated-maps/3_with_bridges/scenario5-2.json, passhum 0, slowness no, forcing stops",52,1,0,0,0,1,0,1,0,0.461,0.071,0.374,0.597,0.425,626,531,932,1367,2830,943.333,14,0.750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"map-generator/generated-maps/3_without_bridges/scenario3-2.json, passhum 0, slowness no, forcing ignoring human",32,1,0,0,0,0,0,1,1,0.770,0.997,0.829,0.077,0.692,706,908,612,588,2108,702.667,22,0.941
"map-generator/generated-maps/3_without_bridges/scenario3-6.json, passhum 0, slowness without rerouting, forcing no",36,0,1,0,0,0,0,1,1,0.531,0.624,0.304,0.037,0.372,706,908,657,550,2115,705.000,19,0.000
"map-generator/generated-maps/3_with_bridges/scenario9-3.json, passhum 0, slowness no, forcing ignoring human",93,1,0,0,0,0,0,1,0,0.645,0.656,0.033,0.038,0.379,891,1360,525,572,2457,819.000,23,0.643
"map-generator/generated-maps/3_with_bridges/scenario8-2.json, passhum 0, slowness without rerouting, forcing change of priorities",82,0,0,1,1,0,0,1,0,0.419,0.074,0.526,0.546,0.457,475,525,1216,1299,3040,1013.333,13,0.000


# Logistic regression (as a baseline)

In [24]:
def split_df_to_X_y(df):
    columns_input_df = [col for col in df.columns
                        if (col if not isinstance(col, str) else col.split('#')[0]) in COLUMNS_INPUT]
    columns_output_df = list(COLUMNS_OUTPUT)
    assert set(COLUMNS_NONSPLIT) | set(columns_input_df) | set(columns_output_df) == set(df.columns)
    
    X = df[columns_input_df]
    y = df[columns_output_df]    
    return X, y


def run_regression(df_train, df_test):
    X_train, y_train = split_df_to_X_y(df_train)
    X_test, y_test = split_df_to_X_y(df_test)
    
    model = sklearn.linear_model.LinearRegression()
    model.fit(X_train, y_train)
    ndarray_predictions = model.predict(X_test)
    df_predictions = pd.DataFrame(ndarray_predictions, columns=y_test.columns)
    return df_predictions


df_predictions_regression = run_regression(df_train, df_test)
df_predictions_regression

,Output of simulation (execution): Total No. of completed missions for AVs (V1-V3),Output of simulation (execution): Collision rate
0,18.371515,0.662266
1,31.396169,0.696378
2,11.584196,1.000203
3,20.529024,0.821077
4,14.646843,0.730966
...,...,...
474,18.766490,1.126028
475,18.129112,0.172816
476,18.133327,0.927477
477,12.547458,0.472452


In [25]:
def save_and_show(fig, basename):  # to avoid inlining large image data into the notebook file
    filename = f'{DIRECTORY_DATA}/{basename}-{random.random()}.png'
    fig.savefig(filename)
    
    # The `random` is because of https://stackoverflow.com/a/43640705.
    display(HTML(f'<img src="{filename}?{random.random()}" alt="{basename}" title="{datetime.datetime.now()}" />'))
    
    plt.close(fig)
    
    return filename


def evaluate_and_plot_column(df_test, df_predictions, column):
    y_test_column = df_test[column]
    predictions_column = df_predictions[column]
    
    r2 = sklearn.metrics.r2_score(y_test_column, predictions_column)
    name = col2parts(column)[1]
    print(f"{name}:")
    print(f"- R^2 Score: {r2}")

    # Plot results for each output column
    fig = plt.figure(figsize=(10, 6))
    plt.scatter(y_test_column, predictions_column, color='blue', alpha=0.5)
    plt.plot([y_test_column.min(), y_test_column.max()], [y_test_column.min(), y_test_column.max()], 'k--', lw=2)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Actual vs Predicted Values for {name}')
    plt.grid(True)
    save_and_show(fig, f'Actual_vs_Predicted_Values_{name}')


def evaluate_and_plot_all_columns(df_test, df_predictions):
    for column in COLUMNS_OUTPUT:
        evaluate_and_plot_column(df_test, df_predictions, column)
        
        
evaluate_and_plot_all_columns(df_test, df_predictions_regression)

Total No. of completed missions for AVs (V1-V3):
- R^2 Score: 0.6382892168352863


Collision rate:
- R^2 Score: 0.36251703004690783


# AutoGluon

In [26]:
def run_autogluon(df_train, df_test):
    X_train, y_train = split_df_to_X_y(df_train)
    X_test, y_test = split_df_to_X_y(df_test)
    
    # Train AutoGluon models
    predictors = []
    df_predictions = pd.DataFrame()
    for column in COLUMNS_OUTPUT:
        print(f'{column=}:')
        df_train_predictor = pd.concat([X_train, y_train[[column]]], axis=1)
        
        predictor = autogluon.tabular.TabularPredictor(
            label=column, 
            eval_metric='r2', 
            problem_type='regression',
        ).fit(
            df_train_predictor,
            presets='medium',  # medium (~1 min.), good (~15 min.), high (~2 h)
            hyperparameters={
                'GBM': {},       # LightGBM (TODO: something like `GBMLarge`)
                'XGB': {},       # XGBoost
                'RF': {},        # Random Forest
                'XT': {},        # Extra Trees
                # 'CAT': {},      # CatBoost, omitted if slow
                # 'NN': {},       # Neural net, if you want it
                # 'LR': {},       # Linear model
                # 'KNN': {},      # K-Nearest Neighbors
            },
        )
        predictors.append(predictor)
        
        df_predictions[column] = predictor.predict(X_test)

        # Leaderboard - Display a table of different models and their performance
        df_test_predictor = pd.concat([X_test, y_test[[column]]], axis=1)
        leaderboard = predictor.leaderboard(df_test_predictor, silent=True)
        show(leaderboard, f'Leaderboard for {column}')
        
        # Feature importance on training data
        # show(
        #     predictor.feature_importance(df_train_predictor),
        #     'feature_importance(df_train_predictor)'
        # )
        # 
        # # Feature importance on test data
        # show(
        #     predictor.feature_importance(df_test_predictor),
        #     'feature_importance(df_test_predictor)'
        # )
        # Example: SHAP values for a specific model
        # shap_values = predictor.get_model_shap_values(df_test_predictor, model='LightGBM')
        # show(shap_values, 'shap_values')  # SHAP values for each feature and each prediction
        
    return predictors, df_predictions


predictors, df_predictions_autogluon = run_autogluon(df_train, df_test)
df_predictions_autogluon

No path specified. Models will be saved in: "AutogluonModels/ag-20250221_091501"
Preset alias specified: 'medium' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.2
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #141~20.04.1-Ubuntu SMP Thu Jan 16 18:38:51 UTC 2025
CPU Count:          16
Memory Avail:       2.30 GB / 31.09 GB (7.4%)
Disk Space Avail:   241.56 GB / 693.60 GB (34.8%)
Presets specified: ['medium']
Beginning AutoGluon training ...
AutoGluon will save models to "/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20250221_091501"
Train Data Rows:    1908
Train Data Columns: 19
Label Column:       Output of simulation (execution): Total No. of completed missions for AVs (V1-V3)
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Avail

column='Output of simulation (execution): Total No. of completed missions for AVs (V1-V3)':


	0.8276	 = Validation score   (r2)
	0.59s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForest ...
	0.7996	 = Validation score   (r2)
	0.47s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: ExtraTrees ...
	0.8031	 = Validation score   (r2)
	0.38s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: XGBoost ...
	0.8223	 = Validation score   (r2)
	0.57s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'LightGBM': 0.667, 'XGBoost': 0.333}
	0.8293	 = Validation score   (r2)
	0.05s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 2.34s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 77346.1 rows/s (382 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20250221_091501")


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.712789,0.829335,r2,0.016687,0.004939,1.217079,0.001818,0.000849,0.053032,2,True,5
1,ExtraTrees,0.708021,0.803119,r2,0.065568,0.036340,0.384972,0.065568,0.036340,0.384972,1,True,3
2,XGBoost,0.707817,0.822320,r2,0.011867,0.002808,0.572449,0.011867,0.002808,0.572449,1,True,4
3,LightGBM,0.707671,0.827579,r2,0.003003,0.001282,0.591598,0.003003,0.001282,0.591598,1,True,1
4,RandomForest,0.691248,0.799589,r2,0.056940,0.045467,0.468137,0.056940,0.045467,0.468137,1,True,2


No path specified. Models will be saved in: "AutogluonModels/ag-20250221_091504"
Preset alias specified: 'medium' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.2
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #141~20.04.1-Ubuntu SMP Thu Jan 16 18:38:51 UTC 2025
CPU Count:          16
Memory Avail:       2.35 GB / 31.09 GB (7.6%)
Disk Space Avail:   241.50 GB / 693.60 GB (34.8%)
Presets specified: ['medium']
Beginning AutoGluon training ...
AutoGluon will save models to "/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20250221_091504"
Train Data Rows:    1908
Train Data Columns: 19
Label Column:       Output of simulation (execution): Collision rate
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    2

column='Output of simulation (execution): Collision rate':


	0.4301	 = Validation score   (r2)
	0.63s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForest ...
	0.1414	 = Validation score   (r2)
	0.52s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: ExtraTrees ...
	0.1343	 = Validation score   (r2)
	0.37s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	0.4721	 = Validation score   (r2)
	0.49s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'XGBoost': 1.0}
	0.4721	 = Validation score   (r2)
	0.05s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 2.28s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 136232.0 rows/s (382 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20250221_091504")


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.480360,0.430115,r2,0.001406,0.000939,0.627751,0.001406,0.000939,0.627751,1,True,1
1,ExtraTrees,0.476534,0.134325,r2,0.070524,0.034746,0.368852,0.070524,0.034746,0.368852,1,True,3
2,RandomForest,0.464370,0.141417,r2,0.061627,0.037110,0.517991,0.061627,0.037110,0.517991,1,True,2
3,XGBoost,0.462954,0.472094,r2,0.010090,0.002156,0.492332,0.010090,0.002156,0.492332,1,True,4
4,WeightedEnsemble_L2,0.462954,0.472094,r2,0.011507,0.002804,0.543226,0.001416,0.000648,0.050894,2,True,5


,Output of simulation (execution): Total No. of completed missions for AVs (V1-V3),Output of simulation (execution): Collision rate
"map-generator/generated-maps/3_with_bridges/scenario3-2.json, passhum 0, slowness with rerouting, forcing 50% stops",18.805965,0.429145
"map-generator/generated-maps/3_without_bridges/scenario6-5.json, passhum 0, slowness no, forcing ignoring human",31.445181,0.461008
"map-generator/generated-maps/3_without_bridges/scenario5-10.json, passhum 0, slowness no, forcing stops",11.576620,0.940489
"map-generator/generated-maps/3_with_bridges/scenario4-4.json, passhum 0, slowness no, forcing ignoring human",19.854589,0.695698
"map-generator/generated-maps/3_with_bridges/scenario5-2.json, passhum 0, slowness no, forcing stops",14.145518,0.830048
...,...,...
"map-generator/generated-maps/3_without_bridges/scenario3-2.json, passhum 0, slowness no, forcing ignoring human",19.631279,0.967266
"map-generator/generated-maps/3_without_bridges/scenario3-6.json, passhum 0, slowness without rerouting, forcing no",16.549858,0.064555
"map-generator/generated-maps/3_with_bridges/scenario9-3.json, passhum 0, slowness no, forcing ignoring human",18.961102,0.796991
"map-generator/generated-maps/3_with_bridges/scenario8-2.json, passhum 0, slowness without rerouting, forcing change of priorities",12.436994,0.358005


## evaluate_and_plot_all_columns

In [27]:
evaluate_and_plot_all_columns(df_test, df_predictions_autogluon)

Total No. of completed missions for AVs (V1-V3):
- R^2 Score: 0.7127894759178162


Collision rate:
- R^2 Score: 0.46295423953150816


## explain_predictions

In [28]:
def explain_predictions(predictors):
    # X_test, y_test = split_df_to_X_y(df_test)
    
    for column, predictor in zip(COLUMNS_OUTPUT, predictors):
        # df_test_predictor = pd.concat([X_test, y_test[[column]]], axis=1)
        if column not in (
            'Output of simulation (execution): Total No. of completed missions for AVs (V1-V3)',
            'Output of simulation (execution): Collision rate',
        ):
            continue
        
        for model in 'LightGBM', 'XGBoost':
            tree_model = predictor._trainer.load_model(model)
            if model == 'LightGBM':
                tree_importance = tree_model.model.feature_importance(importance_type='gain')
            elif model == 'XGBoost':
                tree_importance = tree_model.model.feature_importances_
            else:
                raise ValueError(model)
            # show(tree_importance, column)
                
            
            feature_names = predictor.feature_metadata.get_features()
            df = pd.DataFrame.from_dict(
                {name: {'importance': value} for name, value in zip(feature_names, tree_importance)},
                orient='index'
            )
            # print(df.index)
            # Group indexes by their base name before `#`
            df['group'] = df.index.str.extract(r'^(.+?)(?:#\d+)?$', expand=False)
            df = df.groupby('group')['importance'].sum().to_frame()
            
            df.sort_values(by='importance', ascending=False, inplace=True)
            show(df, f'{model}: {column}')
        
        
explain_predictions(predictors)

,importance
group,
Static map features: No. of OPs,568291.077538
Output of simulation (planning): Total Mission length for AVs (V1-V3),169773.075148
Violation type: Priority violation,43308.096559
Output of simulation (planning): Mission length for AV (V2),40667.924859
Violation type: Priority violation and Speed violation,32155.483060
Output of simulation (planning): Mission length for MV (V0),31485.796543
Output of simulation (planning): Mean POD score for AVs (V1-V3),31250.195447
Output of simulation (planning): Mission length for AV (V1),30419.393932
Coordination strategy: Stops,25667.287539


,importance
group,
Static map features: No. of OPs,0.823102
Output of simulation (planning): Total Mission length for AVs (V1-V3),0.062771
Output of simulation (planning): Mission length for AV (V2),0.015590
Output of simulation (planning): Mission length for AV (V1),0.014447
Output of simulation (planning): Mission length for MV (V0),0.013594
Violation type: Priority violation and Speed violation,0.010824
Output of simulation (planning): Mean POD score for AVs (V1-V3),0.010101
Violation type: Priority violation,0.009462
Violation type: Speed violation,0.009045


,importance
group,
Violation type: Speed violation,606.191634
Output of simulation (planning): POD score for MV (V0),602.876932
Violation type: Priority violation,589.835869
Coordination strategy: Stops,490.714160
Violation type: Priority violation and Speed violation,369.078755
Coordination strategy: Rerouting,274.928040
Coordination strategy: Change of priorities,220.764540
Output of simulation (planning): Mission length for AV (V2),140.940478
Output of simulation (planning): POD score for AV (V2),99.209467


,importance
group,
Violation type: Speed violation,0.277855
Violation type: Priority violation,0.105887
Violation type: Priority violation and Speed violation,0.092920
Output of simulation (planning): POD score for MV (V0),0.073685
Coordination strategy: Stops,0.073173
Coordination strategy: Rerouting,0.064334
Coordination strategy: Change of priorities,0.046164
Output of simulation (planning): Mission length for AV (V2),0.037448
Output of simulation (planning): POD score for AV (V2),0.033508
